**MobileNetV2: Inverted Residuals and Linear Bottlenecks**    
*Mark Sandler, Andrew Howard, Menglong Zhu, Andrey Zhmoginov, Liang-Chieh Chen*   
[[paper](https://arxiv.org/abs/1709.01507)]   
CVPR 2018   